- Preprocessing
- Neural network
- XGboost??
- SVM tuning parameter
- Decision tree
- Stochastic gradient descent

In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from math import floor
stopword = set(stopwords.words('english'))
porter = PorterStemmer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from collections import defaultdict
import pickle
import lime

ImportError: No module named lime

## Preprocessing 

To preprocess, we solve encoding problem and cleaning the text

In [2]:
'awn â'.decode('utf-8').encode('ascii','ignore')
#decode: from string to unicode
#encode: from unicode to string

'awn '

In [ ]:
def tokenize(text):
    text = text.decode('utf-8').encode('ascii', 'ignore')
    words = word_tokenize(text) #split words
    words = [w.lower() for w in words if w.isalpha()] #get rid of punctuation
    words =[w for w in words if  not w in stopword]
    stemmed = [porter.stem(w) for w in words]
    stemmed = ' '.join(w for w in stemmed)
    return stemmed

def create_dataset():
    data = {}
    train = pd.read_csv(r"train.csv")#19579
    test = pd.read_csv(r"test.csv")#8392
    data["submit"] = pd.read_csv(r"sample_submission.csv")
    
    train['Token'] = train.text.map(tokenize)
    test['Token'] = test.text.map(tokenize)
    
    data['valid'] = train.loc[:floor((train1.shape[0]*1)/3)]
    data['train'] = train.loc[floor((train1.shape[0]*1)/3):]
    data['test'] = test
    
    data['valid'] = data['valid'].set_index('id')
    data['test'] = data['test'].set_index('id')
    data['train'] = data['train'].set_index('id')
    
    return data
    



In [20]:
train.head()

,text,author,Token
id,,,
id23567,"He was the devil incarnate, Birch, and I belie...",HPL,devil incarn birch believ eye eye furi could b...
id26623,"We are ready to expose our breasts, exposed te...",MWS,readi expos breast expos ten thousand time bal...
id17046,"He continued, ""You must create a female for me...",MWS,continu must creat femal live interchang sympa...
id26351,And always the goal of my fancies was the migh...,HPL,alway goal fanci mighti vine grown wall littl ...
id13279,"While I remained motionless, and busied in end...",EAP,remain motionless busi endeavor collect though...


In [42]:
classes = ['EAP', 'HPL','MWS']
vectorizer = TfidfVectorizer()

(13053, 3)

In [7]:
#dividing the training set into groups
#hpl = train[train["author"] == "HPL"][['id','token']]
#mws = train[train["author"] == "MWS"][['id','token']]
#eap = train[train["author"] == "eap"][['id','token']]

for training process, we turn each documnet into vector of tfidf. 

* vector.vocabulary_ : a dictionary with keys are list of terms while the values are the id of the according word (e.g: 'fawn': 4296) 
* X_train: a matrix shaper 13053 x 13304 ( # of doc * # of terms), can use X_train[0,:].toarray() to print out each row

In [ ]:
X_valid = vector.transform(list(valid['token']))
X_valid_tfidf= tfidf.transform(X_test)
y_valid = list(valid['author'])

## Simple Naive Bayesian Model 

In [8]:
# TFIDF model
tfidf = TfidfTransformer()

#vector count model
vector= CountVectorizer(decode_error = 'ignore',stop_words = 'english')
# fit and transform the training set
X_train = vector.fit_transform(list(train['Token']))
X_train_tfidf = tfidf.fit_transform(X_train)
y_train = list(train['author'])

#transform the testing set with the training model
X_test = vector.transform(list(valid['Token']))
X_test_tfidf= tfidf.transform(X_test)
y_test = list(valid['author'])

In [9]:
print X_train.shape
print len(y_train)
print X_test.shape
print len(y_test)

(13053, 13304)
13053
(6527, 13304)
6527


In [10]:
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test_tfidf)
float(np.mean(predicted == y_test))

0.8153822583116286

## Support Vector Machine

There are four kernels in SVM: linear kernel, polynomial kernel, RBF(gaussian) kernel, string kernel.For text classification, linear kernel is recommended because:

- Text classification problem is oftern linearly separable
- Large number of feature so mapping a larger space doesn't improve performance 
- Faster
- Less parameters to optimize


In [ ]:
#try linear and rbf kernels with different value of C and gamma
from sklearn.model_selection import GridSearchCV 

parameters = {'kernel':('linear', 'rbf'), 'C':[1,4,7,10], 'gamma': 
              [0.01,0.04,0.10,0.5]}
svr = svm.SVC()
clf_svm1 = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('grid',  GridSearchCV(svr, parameters))])
_ = clf_svm1.fit(list(train['Token']), list(train['author']))
svm_predicted = clf_svm1.predict(list(valid['Token']))
float(np.mean(svm_predicted == y_test))
#C = 1: 0.79
#c= 0.5: 0.791 

In [ ]:
cnf_matrix = confusion_matrix(y_test, predicted)
print(cnf_matrix)

## Stochastic gradient descent

In [ ]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf_svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter = 10000, random_state=42))])

In [ ]:
_= text_clf_svm.fit(list(train['Token']), list(train['author']))
svm_predicted = text_clf_svm.predict(list(valid['Token']))
float(np.mean(svm_predicted == y_test))
#no preprocess: 0.746
#preprocess: 0.758

## Decision Tree Classifier

## Print out the test result 

In [ ]:
test['Token'] = test.text.map(tokenize)
test.head()

In [ ]:
clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
_ = clf.fit(list(train1['Token']), list(train1['author']))
svm_predicted = clf.predict_proba(list(test['Token']))
svm_predicted[0]

In [ ]:
clf.classes_

In [ ]:
eap = [a[0] for a in svm_predicted]
hpl = [a[1] for a in svm_predicted]
mws = [a[2] for a in svm_predicted]

In [ ]:
items = [('id', list(test['id'])), ('EAP',eap),('HPL',hpl),('MWS',mws)]
file_test = pd.DataFrame.from_items(items)
file_test.head()

In [ ]:
file_test.set_index('id')

In [ ]:
file_test.to_csv('result.csv', index = False)

In [ ]:
file_test.shape

In [ ]:
#may think of clean the text first(tokenize), try with SVM next time